In [2]:
import numpy as np
import tensorflow as tf
from keras.datasets import mnist
from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D, Flatten, Dense
from keras.utils import to_categorical
from sklearn.model_selection import train_test_split

In [3]:
# 加载 MNIST 数据集
(x_train, y_train), (x_test, y_test) = mnist.load_data()

In [4]:
# 数据预处理
x_train = x_train.reshape((x_train.shape[0],28,28,1)).astype('float32')/255.0
x_test = x_test.reshape((x_test.shape[0],28,28,1)).astype('float32')/255.0

y_train = to_categorical(y_train,10)
y_test = to_categorical(y_test,10)

**RESHAPE((X_TRAIN.SHAPE[0],28,28,1))**

reshape 是 NumPy 数组的一个方法，用于改变数组的形状。x_train.shape[0] 表示 x_train 数组的第一维大小，也就是样本的数量。(x_train.shape[0],28,28,1) 表示将 x_train 数组重塑为一个四维数组，其中第一维是样本数量，第二维和第三维分别是 28，通常对应图像的高度和宽度，第四维是 1，通常表示图像的通道数（这里可能是灰度图像，只有一个通道）。

**ASTYPE('FLOAT32')**
astype 方法用于将数组的数据类型转换为指定的数据类型。这里将数组的数据类型转换为 float32，因为在深度学习中，通常使用 float32 类型来存储和处理数据，它可以在保证一定精度的同时，减少内存的使用。

**/255.0**

这是一个除法运算，将数组中的每个元素都除以 255.0。在图像数据处理中，图像的像素值通常是在 0 到 255 之间的整数。通过将像素值除以 255.0，可以将像素值归一化到 0 到 1 之间的浮点数，这样可以提高模型的训练效果和稳定性。

**to_categorical(y_train, 10)**

to_categorical(y_train, 10) 是将 y_train 这个标签数组转换为独热编码（One-Hot Encoding）的形式，其中 10 表示类别数量。在机器学习，特别是分类问题中，许多模型如神经网络等，需要将类别标签转换为独热编码的形式，以便更好地进行训练和预测。独热编码可以将离散的类别标签转换为二进制向量，每个向量只有一个元素为 1，其余元素为 0，这样可以避免模型将类别标签误解为具有数值大小关系的数据。例如，在 MNIST 数据集里，y_train 中的标签是 0 - 9 的整数，经过 to_categorical 转换后，每个标签会变成一个长度为 10 的二进制向量，对应位置为 1 表示该样本属于这个类别。

In [5]:
# 划分训练集和验证集
x_train,x_val,y_train,y_val = train_test_split(x_train,y_train,test_size = 0.2,random_state = 42)
# 这里 test_size = 0.2 表示将原始训练数据的 20% 作为验证集，剩下的 80% 作为新的训练集
# 在使用 train_test_split 函数分割数据集时，x_val 通常代表验证集的特征数据。
# 在机器学习中，为提高模型的泛化能力、防止过拟合，常将整体数据划分为训练集、验证集和测试集三部分。
# 训练集用于模型训练，验证集用于在训练过程中评估模型性能，调整模型的超参数，而测试集则用于最终评估模型的泛化能力。

# 当使用 train_test_split 函数进行两次划分时，第一次将整体数据划分为训练集和测试集，第二次将第一次划分得到的训练集进一步划分为新的训练集和验证集。

**超参数的定义**

超参数是指在机器学习模型学习过程中需要手动设置的参数，这些参数并不直接通过训练数据学习而来，而是在训练之前确定。常见的超参数包括学习率、正则化系数、树的深度、树的数量等，它们对模型的表现有着显著影响。

**超参数的作用**

超参数的选择对模型的性能有重要影响。通过合理地选择和调整超参数，可以显著提高模型的准确性和泛化能力。例如，学习率控制着模型在训练过程中参数更新的步长，合适的学习率能使模型更快收敛到最优解；正则化系数可以防止模型过拟合，提高模型的泛化能力。

**超参数的调整方法**

在开始超参数调整之前，需要根据具体问题定义一个目标函数并确定超参数的取值范围。超参数的取值范围需要根据具体任务和模型进行设置，通常可以参考相关论文或经验值，并结合试错的方式逐步调整。对于一些连续型超参数，可以考虑使用对数均匀分布或对数正态分布等非线性取值范围。然后，使用所选的超参数调整方法来进行搜索，并在最后评估超参数的性能。如果发现性能不足，则可以继续尝试不同的超参数组合，直到获得最优的结果为止。

In [9]:
# 构建 CNN 模型
model = Sequential([
    Conv2D(32,kernel_size = (3,3),activation = 'relu',input_shape = (28,28,1)),
    # 32：表示该卷积层使用 32 个卷积核（滤波器），每个卷积核会在输入数据上滑动，提取不同的特征。
    # kernel_size=(3, 3)：卷积核的大小为 3x3。
    # input_shape=(28, 28, 1)：指定输入数据的形状，这里表示输入的是 28x28 像素的单通道（灰度图）图像。
    MaxPooling2D(pool_size = (2,2)),
    # 最大池化层用于对卷积层的输出进行下采样，pool_size=(2, 2) 表示使用 2x2 的池化窗口，在每个 2x2 的区域中取最大值作为该区域的输出。这样可以减少数据的维度，降低计算量，同时保留重要的特征。
    Conv2D(64,kernel_size = (3,3),activation = 'relu'),
    # 后续的卷积层类似，分别使用 64 和 128 个 3x3 的卷积核，激活函数同样为 ReLU。随着网络的深入，卷积核的数量增加，模型可以学习到更高级、更复杂的特征。
    MaxPooling2D(pool_size = (2,2)),
    # 最大池化层用于对卷积层的输出进行下采样，pool_size=(2, 2) 表示使用 2x2 的池化窗口，在每个 2x2 的区域中取最大值作为该区域的输出。这样可以减少数据的维度，降低计算量，同时保留重要的特征。
    Conv2D(128,kernel_size = (3,3),activation = 'relu'),
    # 后续的卷积层类似，分别使用 64 和 128 个 3x3 的卷积核，激活函数同样为 ReLU。随着网络的深入，卷积核的数量增加，模型可以学习到更高级、更复杂的特征。
    Flatten(),
    # 将前面卷积层和池化层输出的多维数据展平为一维向量，以便输入到全连接层。
    Dense(128,activation = 'relu'),
    # 该全连接层包含 128 个神经元，激活函数为 ReLU，用于对展平后的特征进行进一步的处理和学习。
    Dense(10,activation = 'softmax')
    # 最后一层是输出层，包含 10 个神经元，使用 softmax 作为激活函数。softmax 函数将输出转换为概率分布，适用于多分类问题，这里假设是一个 10 类的分类任务。
])

In [12]:
# 编译模型
model.compile(
    optimizer = 'adam', # 使用 adam 优化器
    loss = 'categorical_crossentropy', # 使用 categorical_crossentropy 作为损失函数，因为这是多类分类问题。
    metrics = ['accuracy']
)

In [13]:
# 训练模型
history = model.fit(x_train,y_train,epochs = 10,batch_size = 32,validation_data = (x_val,y_val))
# epochs = 10：epochs 参数指定了训练过程中模型将对整个训练数据集进行迭代的次数。在这个例子中，模型将对 x_train 和 y_train 进行 10 次完整的遍历。每次遍历都会更新模型的权重，以逐步提高模型的性能。
# batch_size = 32：batch_size 参数定义了在每次梯度更新时使用的样本数量。在训练过程中，训练数据会被分成多个小批量（batch），模型会依次对每个小批量的数据进行训练并更新权重。这里，每个小批量包含 32 个样本。较小的 batch_size 可以使模型在训练过程中更频繁地更新权重，可能有助于模型收敛到更好的解，但也会增加训练时间；较大的 batch_size 可以加快训练速度，但可能会导致模型陷入局部最优解。
# validation_data = (x_val,y_val)：validation_data 参数用于指定验证数据集。它是一个元组，包含验证数据的输入特征 x_val 和对应的目标标签 y_val。在每个 epoch 结束后，模型会在验证数据集上进行评估，计算验证损失和验证指标（如准确率等）。这有助于监控模型在未见过的数据上的性能，判断模型是否过拟合。

# history：model.fit 方法会返回一个 History 对象，该对象包含了训练过程中的详细信息，如每个 epoch 的训练损失、训练指标、验证损失和验证指标等。可以通过 history.history 属性访问这些信息，例如 history.history['loss'] 可以获取每个 epoch 的训练损失。

Epoch 1/10
1500/1500 ━━━━━━━━━━━━━━━━━━━━ 34s 22ms/step - accuracy: 0.8962 - loss: 0.3278 - val_accuracy: 0.9825 - val_loss: 0.0531
Epoch 2/10
1500/1500 ━━━━━━━━━━━━━━━━━━━━ 41s 27ms/step - accuracy: 0.9840 - loss: 0.0501 - val_accuracy: 0.9832 - val_loss: 0.0569
Epoch 3/10
1500/1500 ━━━━━━━━━━━━━━━━━━━━ 41s 28ms/step - accuracy: 0.9911 - loss: 0.0292 - val_accuracy: 0.9863 - val_loss: 0.0449
Epoch 4/10
1500/1500 ━━━━━━━━━━━━━━━━━━━━ 43s 29ms/step - accuracy: 0.9929 - loss: 0.0213 - val_accuracy: 0.9905 - val_loss: 0.0326
Epoch 5/10
1500/1500 ━━━━━━━━━━━━━━━━━━━━ 49s 33ms/step - accuracy: 0.9952 - loss: 0.0148 - val_accuracy: 0.9878 - val_loss: 0.0446
Epoch 6/10
1500/1500 ━━━━━━━━━━━━━━━━━━━━ 45s 30ms/step - accuracy: 0.9956 - loss: 0.0122 - val_accuracy: 0.9904 - val_loss: 0.0350
Epoch 7/10
1500/1500 ━━━━━━━━━━━━━━━━━━━━ 45s 30ms/step - accuracy: 0.9963 - loss: 0.0116 - val_accuracy: 0.9895 - val_loss: 0.0453
Epoch 8/10
1500/1500 ━━━━━━━━━━━━━━━━━━━━ 52s 35ms/step - accuracy: 0.9969 -

In [14]:
# 评估模型
test_loss,test_acc = model.evaluate(x_test,y_test,verbose = 2)
print(f'Test accuracy: {test_acc}')

# 代码 test_loss, test_acc = model.evaluate(x_test, y_test, verbose = 2) 的主要功能是使用测试数据集评估已经训练好的模型的性能。下面是对代码各部分的详细解释：

# model.evaluate()：这是 Keras 模型对象的一个方法，用于在给定的测试数据上评估模型。该方法会返回模型在测试数据上的损失值和指定的评估指标值。
# x_test：代表测试数据的输入特征，通常是一个 NumPy 数组或张量，包含了用于评估模型的样本特征信息。
# y_test：代表测试数据的真实标签，同样通常是一个 NumPy 数组或张量，对应于 x_test 中每个样本的真实输出值。
# verbose = 2：verbose 参数用于控制评估过程的信息显示方式。verbose = 2 表示每一个测试批次结束后，会以进度条的形式显示评估进度。
# 代码会返回两个值：test_loss 和 test_acc。
# test_loss 是模型在测试数据上的损失值，损失值衡量了模型预测结果与真实标签之间的差异程度，损失值越小，说明模型的预测结果越接近真实标签。
# test_acc 是模型在测试数据上的准确率，准确率表示模型正确预测的样本数占总样本数的比例，准确率越高，说明模型的性能越好。

313/313 - 4s - 13ms/step - accuracy: 0.9911 - loss: 0.0405
Test accuracy: 0.991100013256073


In [15]:
# 预测
predictions = model.predict(x_test)

313/313 ━━━━━━━━━━━━━━━━━━━━ 4s 12ms/step
